In [9]:

keyword='养发|沐发|沐头|洗发|涂之立生|润发|立生|脱发|长发|发落'
import numpy as np 
import pandas as pd 
import glob
import os
filelist=sorted(glob.glob("/content/drive/MyDrive/NatureLanguageProgram/NLP_final_project/ChineseMedicalBook/*.txt"))

def agent(filelist):
  reader=pd.read_csv(filelist,header=None,sep='\r',engine='python',error_bad_lines=False)
  reader.columns=['data']
  reader['book']=reader['data'][0]
  reader['item']=reader['data'].apply(lambda x: x if '<篇名>' in x else None  )
  reader['item']=reader['item'].fillna(method='ffill')
  reader['keyWords']=reader['data'].str.contains(keyword)
  temp=reader[reader['keyWords']==True]
  return temp[['data','item','book']]

tempall=pd.DataFrame(agent(filelist[0]))
for i in range(681):
    temp=agent(filelist[i])
    tempall=tempall.append(temp,ignore_index=True,)   
tempall.to_csv('output1.csv',index=None) 
tempall

IndexError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
from opencc import OpenCC
cc = OpenCC('s2t')
path = "ChineseMedicalBook"#資料夾目錄
files= os.listdir(path) #得到資料夾下的所有檔名稱
# print(type(files))
# print(files)
store = open('zh','w')
for file in files:
    file=open(file,'r',encoding='utf-8')
    txt = []
    for line in file.readlines():
        txt.append(line)
    for data in txt:
        data = cc.convert(data)
    store.write(data)

In [ ]:

import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split    #分割資料集

def cutval(df):
    topcut = []
    for d in df['劇情介紹']:
        top = jieba.analyse.extract_tags(d,topK=2)
        topcut.append(top)
    typecut = []
    for d in df['oneLabel']:
        # ch =re.compile("[\u4e00-\u9fa5]")
        # seg_word =  "".join(ch.findall(d))
        # top = jieba.lcut(seg_word)
        typecut.append(d)
    # print(typecut)
    namecut = []
    for d in df['中文名稱']:
        ch =re.compile("[\u4e00-\u9fa5]")
        name =  "".join(ch.findall(d))
        # print("name = ")
        # print(name )
        namecut.append(name)
    # print(namecut)
    data = { 'type':typecut ,'name':namecut,'article':topcut}
    df1 = pd.DataFrame(data)

    df1['type'] = df1['type'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
    df1['article'] = df1['article'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
    df1['key'] = df1['name'].astype(str)+' '+df1['type'].astype(str)+' '+df1['article'].astype(str)
    return df1

def count(df):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['key'])
    tfidf = TfidfTransformer() 
    tf=tfidf.fit_transform(X)
    word = vectorizer.get_feature_names() #詞袋 
    
    return tf

def knn(X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y['type'].str[0:2], test_size=0.07)
    clf=KNeighborsClassifier(n_neighbors=51)
    clf.fit(X_train,y_train)

    y_pred = clf.predict(X_test)    # 預測模型 
    y_test=y_test.values

    print("精準度：",metrics.accuracy_score(y_test, y_pred))

df = pd.read_csv('movies.csv')
df = df.drop(labels=['Unnamed: 0'],axis='columns')
df1 = cutval(df) 
tf = count(df1)
knn(tf,df1)